In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from bs4 import BeautifulSoup as soup
import requests
from lxml import html
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path2 = "/content/drive/My Drive/California Wildfire Predictor/Data/California_Fires.csv"
CF = pd.read_csv(path2)
CF.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06T18:30:00Z,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08T18:30:00Z,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-07-30T18:00:00Z,NaN,False,True,NaN,NaN,NaN,33.709500,Hwy 243 & Hwy 74 near Mountain Center,-116.728850,False,Mountain Fire,100.0,NaN,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,31,NaN,NaN,NaN,2013-08-30T08:00:00Z,NaN,False,True,NaN,NaN,NaN,39.120000,"Deadwood Ridge, northeast of Foresthill",-120.650000,False,American Fire,100.0,NaN,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,56,47.0,8.0,117.0,2013-05-11T06:30:00Z,NaN,False,True,NaN,11.0,10.0,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,0.000000,True,Springs Fire,100.0,2167.0,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [ ]:
#cleaning the data for unrealistic coordinates 
cleaned_CF = CF[(CF.Longitude<-115) & (CF.Latitude<44) & (CF.Latitude > 30)]
cleaned_CF = cleaned_CF[cleaned_CF.CanonicalUrl!='/incidents/2013/8/6/tram-fire/']
# Create a new column to turn 0 acresburned to 1 acre burned (for visualization purposes)
cleaned_CF["Fixed0AcresBurned"] = cleaned_CF["AcresBurned"]
cleaned_CF.loc[cleaned_CF["Fixed0AcresBurned"] == 0, "Fixed0AcresBurned"] = 1
cleaned_CF[cleaned_CF["Fixed0AcresBurned"] == 1].head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders,Fixed0AcresBurned
160,0.0,False,Six Rivers National Forest,NaN,2013,False,/incidents/2013/8/3/orleans-complex/,NaN,NaN,Humboldt,12,NaN,NaN,NaN,2013-08-03T15:00:00Z,NaN,False,True,NaN,NaN,NaN,41.333000,"10 miles east of Somes Bar, California",-123.376000,False,Orleans Complex,100.0,NaN,True,The Orleans Complex burned in August 2013. 10 ...,"Orleans Complex, August 2013, Somes Bar, Six R...",2013-08-03T08:00:00Z,Finalized,NaN,NaN,NaN,NaN,7d4f107b-66a9-43d2-819d-ec855c2947bf,2013-08-03T15:00:00Z,NaN,1.0
161,0.0,False,USFS San Bernardino National Forest,NaN,2013,False,/incidents/2013/6/9/hathaway-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-06-09T16:45:00Z,NaN,False,True,NaN,NaN,NaN,33.996470,"Hathaway Canyon area, northeast of Banning",-116.841370,False,Hathaway Fire,100.0,NaN,True,The Hathaway Fire burned in June 2013 in the H...,"Hathaway Fire, June 2013, USFS San Bernardino ...",2013-06-09T12:30:00Z,Finalized,NaN,NaN,NaN,NaN,befb9d5c-c0ea-48b8-8589-2586be0275af,2013-06-09T16:45:00Z,NaN,1.0
278,0.0,False,US Forest Service - Klamath National Forest,NaN,2014,False,/incidents/2014/8/11/happy-camp-complex/,NaN,NaN,Siskiyou,47,NaN,NaN,NaN,2014-08-14T09:30:00Z,NaN,False,True,NaN,NaN,NaN,41.796000,in the Happy Camp area,-123.374000,False,Happy Camp Complex,100.0,NaN,True,"The Happy Camp Complex started on August 11, 2...","Happy Camp Complex, Siskiyou County, US Forest...",2014-08-11T22:00:00Z,Finalized,NaN,NaN,NaN,NaN,da500577-e558-4446-832f-0e537cc55204,2014-08-14T09:30:00Z,NaN,1.0
430,0.0,False,US Forest Service - Humboldt - Toiyabe Nationa...,NaN,2015,False,/incidents/2015/6/19/washington-fire/,NaN,NaN,Alpine,2,NaN,NaN,NaN,2015-06-22T12:05:00Z,NaN,False,True,NaN,NaN,NaN,38.598056,south of Markleeville,-119.751944,False,Washington Fire,100.0,NaN,True,"The Washington Fire started on June 19, 2015 ...","Washington Fire, Alpine County, Markleeville, ...",2015-06-19T19:00:00Z,Finalized,NaN,NaN,NaN,NaN,470d19d4-384d-497a-9770-cd2e2a3f3196,2015-06-22T12:05:00Z,NaN,1.0
610,0.0,False,Stanislaus National Forest,NaN,2016,False,/incidents/2016/8/18/mokelumne-fire/,NaN,NaN,Alpine,2,NaN,NaN,NaN,2016-08-30T12:05:00Z,NaN,False,True,NaN,NaN,NaN,38.575540,Mokelumne Wilderness,-120.006060,False,Mokelumne Fire,100.0,NaN,True,"The Mokelumne Fire started on August 18, 2016 ...","Mokelumne Fire, Alpine County, August 18, 2016...",2016-08-18T12:05:00Z,Finalized,NaN,NaN,NaN,NaN,5823867d-c458-4673-858f-0ded99b6121f,2016-08-30T12:05:00Z,NaN,1.0


In [ ]:
zoom_factor = 3 
radius_scaling = 1 #scale of fire bubble size 

base_map = folium.Map(location=[36,-120], zoom_start=zoom_factor, tiles = None)
folium.TileLayer('Open Street Map', control=False).add_to(base_map)
for year, year_grp in cleaned_CF.groupby('ArchiveYear'):
    feature_group = folium.FeatureGroup(year)
    for row in year_grp.itertuples():
      folium.CircleMarker(
        location=[row.Latitude, row.Longitude],
        radius= (row.Fixed0AcresBurned)*0.00015,
        color='red',
        popup='CanonicalUrl:' + row.CanonicalUrl + ' - Year:' + str(int(row.ArchiveYear)) + ' - Acres Burned:' 
        + str(row.AcresBurned),
        fill=True,
        fill_color='red' 
        ).add_to(feature_group)
    feature_group.add_to(base_map)

folium.LayerControl().add_to(base_map)

title_html = '''
             <h3 align="center" style="font-size:40px"><b>Californian Wildfires (2013-2019)</b></h3>
             '''
base_map.get_root().html.add_child(folium.Element(title_html))
base_map